In [5]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [6]:
class LiveTrade:
    
    def __init__(self):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        
        rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 24000)
        nf = pd.DataFrame(rate)


        nf['date']=pd.to_datetime(nf['time'], unit='s')

        nf.set_index(np.arange(len(nf)), inplace = True)
        nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


        self.df = nf.iloc[:500]  # First half
        self.df2 = nf.iloc[500:]  # Second half
        self.df['squeeze_on'] = False
        
        self.prev_high, self.prev_low, self.current_high, self.current_low = float(), float(), float(), float()

        self.trend_date  = None
        self.prev_trend = None
        
        self.bull_no_trend = False
        self.bear_no_trend = False

        self.no_trend_up = False
        self.no_trend_down = False
        
        self.past_trend()
        
    
    def past_trend(self):
        length = len(self.df)

        # Calculate max and min high values
        self.prev_high = self.df.iloc[length - 42 : length - 21]['high'].max()
        self.prev_low = self.df.iloc[length - 42 : length - 21]['low'].min()

        self.current_high = self.df.iloc[length - 21 : length]['high'].max()
        self.current_low = self.df.iloc[length - 21 : length]['low'].min()

        # Up condition
        if self.current_high > self.prev_high and self.current_low > self.prev_low:
            self.df.loc[length - 21 : length, 'trend'] = 'up'
        # Down condition
        elif self.current_high < self.prev_high and self.current_low < self.prev_low:
            self.df.loc[length - 21 : length, 'trend'] = 'down'
        else:
            self.df.loc[length - 21 : length, 'trend'] = 'no trend'

        self.prev_high = self.current_high
        self.prev_low = self.current_low
        self.trend_date = self.df['date'].iloc[-1]
        self.prev_trend = self.df['trend'].iloc[-1]
    
    
    def test_current_trend(self, high, low, l):
        
        if self.prev_trend == 'up' and self.bull_no_trend is False:
            
            if low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
                
            else:
                self.df.loc[l, 'trend'] = 'no trend bull'
                self.bull_no_trend = True
        
        elif self.prev_trend == 'up' and self.bull_no_trend is True:
            self.df.loc[l, 'trend'] = 'no trend bull 1'
        
        elif self.prev_trend == 'down' and self.bear_no_trend is False:
            
            if high < self.prev_high:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend bear'
                self.bear_no_trend = True
        
        elif self.prev_trend == 'down' and self.bear_no_trend is True:
            self.df.loc[l, 'trend'] = 'no trend bear1'
        
        elif self.prev_trend == 'no trend' and self.no_trend_up is False and self.no_trend_down is False:
            
            if high > self.prev_high:
                self.df.loc[l, 'trend'] = 'up'
                self.no_trend_up = True
            
            elif low < self.prev_low:
                self.df.loc[l, 'trend'] = 'down'
                self.no_trend_down = True
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
        
        elif self.prev_trend == 'no trend' and self.no_trend_up is True:
            
            if low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
                self.no_trend_up = False
        
        elif self.prev_trend == 'no trend' and self.no_trend_down is True:
            
            if high < self.prev_high:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
                self.no_trend_down = False

    def calculate_current_trend(self, position_index, days=21):
        l = len(self.df) - 1

        if position_index + days > l:
            self.test_current_trend(self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
        
        elif position_index + days == l:
            current_high = self.df.loc[self.df.index[position_index: l + 1], 'high'].max()
            current_low = self.df.loc[self.df.index[position_index: l + 1], 'low'].min()
            
            if current_high > self.prev_high and current_low > self.prev_low:
                self.df.loc[l, 'trend'] = 'up'
            
            elif current_high < self.prev_high and current_low < self.prev_low:
                self.df.loc[l, 'trend'] = 'down'
            
            else:
                self.df.loc[l, 'trend'] = 'no trend'
            
            self.trend_date = self.df['date'].iloc[-1]
            self.prev_high = current_high
            self.prev_low = current_low
            self.prev_trend = self.df['trend'].iloc[-1]
            self.bull_no_trend, self.bear_no_trend, self.no_trend_up, self.no_trend_down = [False] * 4
            
            
            
    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))

        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]

    def calculate_indicators(self):
        length = len(self.df) - 1

        self.df.loc[length, 'EMA_8'] = talib.EMA(self.df['close'], timeperiod=8).iloc[-1]
        self.df.loc[length, 'EMA_200'] = talib.EMA(self.df['close'], timeperiod=200).iloc[-1]
        self.df.loc[length, 'RSI'] = talib.RSI(self.df['close'], timeperiod=21).iloc[-1]

        sma = self.df['close'].rolling(window=20).mean()
        stddev = self.df['close'].rolling(window=20).std()
        lower_band = sma - (2 * stddev)
        upper_band = sma + (2 * stddev)

        kc_middle, kc_upper, kc_lower = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback=20, multiplier=1.2, atr_lookback=10)

        if (lower_band.iloc[-1] > kc_lower) and (upper_band.iloc[-1] < kc_upper):
            self.df.loc[length, 'squeeze_on'] = True
        else:
            self.df.loc[length, 'squeeze_on'] = False

        self.df.loc[length, 'EMA_20'], self.df.loc[length, 'upper_keltner'], self.df.loc[length, 'lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)

    
    
    def Caller(self):
        # Initialize an empty DataFrame to store the past data that gets dropped
        self.past_df = pd.DataFrame()

        for index, row in self.df2.iterrows():
            # Add live data to the df DataFrame
            self.df = self.df.append(row, ignore_index=True)

            # Delete the first row if the DataFrame has more than 202 rows
            if len(self.df) > 500:
                # Save the dropped row in past_df
                self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
                self.df = self.df.iloc[1:].reset_index(drop=True)

                
            position_index = self.df[self.df['date'] == self.trend_date].index[0]
    
            self.calculate_current_trend(position_index)
            # Calculate indicators for the current state of df
            self.calculate_indicators()

        # Concatenate past_df and df to get the full dataset
        self.result_df = pd.concat([self.past_df, self.df], ignore_index=True)

        # Save the result_df to an Excel file
        self.result_df.to_excel('combined.xlsx')


In [7]:
ob = LiveTrade()

ob.Caller()

C:\Users\hp\AppData\Local\Temp\ipykernel_6584\1536449314.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['squeeze_on'] = False
C:\Users\hp\AppData\Local\Temp\ipykernel_6584\1536449314.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.loc[length - 21 : length, 'trend'] = 'up'
C:\Users\hp\AppData\Local\Temp\ipykernel_6584\1536449314.py:197: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = se